In [1]:
## import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, Activation, Bidirectional
import tensorflow as tf

Using TensorFlow backend.
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lab606a/.virtualenvs/keras/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lab606a/.virtualenvs

In [2]:
# set GPU memory
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
datasetset = pd.read_csv("./datasets/split by random for distributed/classification/increasing_not_fixed_dataset.csv", header=None)
datasetset = datasetset.fillna(0)
datasetset = np.array(datasetset)
datasetset.shape

(104000, 91)

In [4]:
x_train = datasetset[:,1:]
x_train.shape

(104000, 90)

In [5]:
row = int(x_train.shape[1]/3)
col = 3

In [6]:
x_train = x_train.reshape(x_train.shape[0], row, col)
x_train = x_train.astype('float32')
x_train.shape

(104000, 30, 3)

In [7]:
y_train = datasetset[:,0]
y_train = y_train.astype('int')
y_train.shape

(104000,)

In [8]:
n_classes = 8

In [9]:
y_train = np_utils.to_categorical(y_train, n_classes)
y_train.shape

(104000, 8)

In [10]:
model = Sequential()
model.add(CuDNNLSTM(units=256, input_shape=(row,col), return_sequences=True))
model.add(CuDNNLSTM(units=128, return_sequences=True))
model.add(CuDNNLSTM(units=64, return_sequences=True))
model.add(CuDNNLSTM(units=32, return_sequences=True))
model.add(CuDNNLSTM(units=16, return_sequences=True))
model.add(CuDNNLSTM(units=8))
model.add(Dense(units=n_classes, activation='softmax'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 30, 256)           267264    
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 30, 128)           197632    
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 30, 64)            49664     
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 30, 32)            12544     
_________________________________________________________________
cu_dnnlstm_5 (CuDNNLSTM)     (None, 30, 16)            3200      
_________________________________________________________________
cu_dnnlstm_6 (CuDNNLSTM)     (None, 8)                 832       
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
Total para

In [12]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
# training
batch_size = 8000
training_iters = 500
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=training_iters, shuffle=True)

Epoch 1/500
104000/104000 [==============================] - 4s 35us/step - loss: 2.0191 - acc: 0.1997
Epoch 2/500
104000/104000 [==============================] - 3s 27us/step - loss: 1.9713 - acc: 0.2308
Epoch 3/500
104000/104000 [==============================] - 3s 26us/step - loss: 1.8705 - acc: 0.3043
Epoch 4/500
104000/104000 [==============================] - 3s 26us/step - loss: 1.7870 - acc: 0.3497
Epoch 5/500
104000/104000 [==============================] - 3s 27us/step - loss: 1.6834 - acc: 0.3943
Epoch 6/500
104000/104000 [==============================] - 3s 26us/step - loss: 1.5842 - acc: 0.4338
Epoch 7/500
104000/104000 [==============================] - 3s 27us/step - loss: 1.5240 - acc: 0.4567
Epoch 8/500
104000/104000 [==============================] - 3s 27us/step - loss: 1.4489 - acc: 0.4996
Epoch 9/500
104000/104000 [==============================] - 3s 27us/step - loss: 1.3759 - acc: 0.5447
Epoch 10/500
104000/104000 [==============================] - 3s 27us/ste

104000/104000 [==============================] - 3s 28us/step - loss: 0.0716 - acc: 0.9814
Epoch 158/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.1712 - acc: 0.9474
Epoch 159/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.2124 - acc: 0.9318
Epoch 160/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.1224 - acc: 0.9634
Epoch 161/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0780 - acc: 0.9801
Epoch 162/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.0616 - acc: 0.9856
Epoch 163/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.0556 - acc: 0.9876
Epoch 164/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.0520 - acc: 0.9891
Epoch 165/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.0469 - acc: 0.9906
Epoch 166/500
104000/104000 [==============================] - 3s 27u

104000/104000 [==============================] - 3s 28us/step - loss: 0.0327 - acc: 0.9917
Epoch 314/500
104000/104000 [==============================] - 3s 27us/step - loss: 1.1308 - acc: 0.7427
Epoch 315/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.5080 - acc: 0.8429
Epoch 316/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.2017 - acc: 0.9335
Epoch 317/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.1031 - acc: 0.9677
Epoch 318/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0621 - acc: 0.9832
Epoch 319/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0509 - acc: 0.9870
Epoch 320/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0445 - acc: 0.9886
Epoch 321/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0358 - acc: 0.9923
Epoch 322/500
104000/104000 [==============================] - 3s 28u

104000/104000 [==============================] - 3s 27us/step - loss: 0.0097 - acc: 0.9990
Epoch 470/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0096 - acc: 0.9991
Epoch 471/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0091 - acc: 0.9991
Epoch 472/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0088 - acc: 0.9992
Epoch 473/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0087 - acc: 0.9991
Epoch 474/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.0081 - acc: 0.9993
Epoch 475/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0081 - acc: 0.9993
Epoch 476/500
104000/104000 [==============================] - 3s 28us/step - loss: 0.0077 - acc: 0.9994
Epoch 477/500
104000/104000 [==============================] - 3s 27us/step - loss: 0.0076 - acc: 0.9993
Epoch 478/500
104000/104000 [==============================] - 3s 28u

In [30]:
def cal_acc(direction, speed):
    speed = str(speed)
    file = './datasets/classification/' +  direction + speed + '_test.csv'
    test_up2 = pd.read_csv(file, header=None)
    test_up2 = test_up2.fillna(0)
    test_up2 = np.array(test_up2)
    test_up2 = test_up2.reshape(test_up2.shape[0],row,col)
    cnt = np.array([0,0,0,0,0,0,0,0])
    pred = model.predict(test_up2)
    for i in range (pred.shape[0]):
        for j in range (8):
            if max(pred[i,:]) == pred[i,j]:
                cnt[j] += 1
    return cnt

In [39]:
accuarcy = np.zeros([8,8])
accuarcy[0,:] = cal_acc('top', 5)
accuarcy[1,:] = cal_acc('top', 6)
accuarcy[2,:] = cal_acc('left', 5)
accuarcy[3,:] = cal_acc('left', 6)
accuarcy[4,:] = cal_acc('right', 5)
accuarcy[5,:] = cal_acc('right', 6)
accuarcy[6,:] = cal_acc('back', 5)
accuarcy[7,:] = cal_acc('back', 6)
accuarcy

array([[250.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0., 249.,   0.,   0.,   1.,   0.,   0.,   0.],
       [  0.,   0., 248.,   2.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0., 249.,   0.,   1.,   0.,   0.],
       [  0.,   1.,   0.,   0., 244.,   5.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   4., 245.,   0.,   1.],
       [  0.,   0.,   0.,   0.,   0.,   1., 246.,   3.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0., 250.]])

In [32]:
top5_test = pd.read_csv('./datasets/classification/top5_test.csv', header=None)
top5_test = np.array(top5_test)
top5_test.shape

(250, 90)

In [33]:
np.set_printoptions(suppress=True)
for i in range(14,90,3):
    tmp = top5_test[1,:i]
    tmp = tmp.reshape(1, tmp.shape[0])
    tmp = sequence.pad_sequences(tmp, maxlen=(row*col), padding='post', dtype='float32')
    tmp = tmp.reshape(1, 30, 3)
    print((i+1)/3)
    preddd = model.predict(tmp, verbose=1)
    print(preddd)

5.0
1/1 [==============================] - 0s 15ms/step
[[0.0029166  0.11210812 0.00054193 0.73945653 0.00028281 0.02691992
  0.00072269 0.11705127]]
6.0
1/1 [==============================] - 0s 2ms/step
[[0.00001487 0.99915946 0.00002926 0.00022902 0.00006214 0.00041232
  0.00000419 0.00008874]]
7.0
1/1 [==============================] - 0s 3ms/step
[[0.00001086 0.9994247  0.00001837 0.00011779 0.0000519  0.00031207
  0.00000262 0.00006166]]
8.0
1/1 [==============================] - 0s 3ms/step
[[0.00001235 0.99941325 0.00001869 0.00013078 0.00005132 0.00030577
  0.00000287 0.00006495]]
9.0
1/1 [==============================] - 0s 3ms/step
[[0.00002366 0.99937326 0.0000308  0.00016646 0.00007467 0.00026145
  0.00000385 0.00006582]]
10.0
1/1 [==============================] - 0s 3ms/step
[[0.00056837 0.99776626 0.00037969 0.00059166 0.00046696 0.00013966
  0.00001495 0.00007247]]
11.0
1/1 [==============================] - 0s 3ms/step
[[0.08658595 0.8906276  0.00729693 0.01258123 0.

In [34]:
#model.save('./classification_notfixed_20200505_256to8')